# Environment

In [2]:
import sys
from os import path
sys.path.append(path.dirname(path.dirname(path.abspath("Test.ipynb"))))
import time
import logging
from Controllers_Definition import testcontroller1, testcontroller2, testcontroller3, testcontroller4, timeoutcontroller
from datetime import datetime
from FMLC.triggering import triggering
from FMLC.baseclasses import eFMU
from FMLC.stackedclasses import controller_stack
import random
import threading
import multiprocessing as mp

In [3]:
logger = logging.getLogger(__name__)
'''
import matplotlib.pyplot as plt
%matplotlib inline
'''
logger.setLevel(logging.DEBUG)

# Triggering Class

In [3]:
ts = {} 
ts['main'] = 0.5 # seconds
ts['print'] = 1 # seconds

trigger_test = triggering(ts)
now_init = time.time()
now = now_init
while now < now_init+2:
    now = time.time()
    if now >= trigger_test.trigger['main']:
        print ('Main triggered', now)
        trigger_test.refresh_trigger('main', now)
    if now >= trigger_test.trigger['print']:
        print ('Print triggered', now)
        trigger_test.refresh_trigger('print', now)

Main triggered 1615270972.5
Main triggered 1615270973.000362
Print triggered 1615270973.000362
Main triggered 1615270973.5


# Controller Base Class (eFMU)

In [4]:
# Test controller
testcontroller = testcontroller1() #this controller multiplies the inputs
# Get all variables
variables = testcontroller.get_model_variables()
# Makeup some inputs
inputs = {}
for var in variables:
    inputs[var] = random.randint(1,50)
# Query controller
print ('Log-message', testcontroller.do_step(inputs=inputs))
print ('Input', testcontroller.input)
print ('Output', testcontroller.output)
print('Output', testcontroller.get_var('output'))

Log-message testcontroller1 did a computation!
Input {'a': 13, 'b': 6}
Output {'c': 78}
Output {'c': 78}


# Controller Stack Class (single-thread/multi-thread)

In [4]:
# create a mapping of controllers and their sample times
controllers = {}
controllers['forecast1'] = {'fun':testcontroller1, 'sampletime':1}
controllers['mpc1'] = {'fun':testcontroller2, 'sampletime':'forecast1'}
controllers['control1'] = {'fun':testcontroller3, 'sampletime':'mpc1'}
controllers['forecast2'] = {'fun':testcontroller3, 'sampletime':1}
controllers['forecast3'] = {'fun':testcontroller1, 'sampletime':1}

# Create a mapping of inputs for each controller
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'

## Single Thread

In [6]:
# Initialize the controller_stack using the mappings above
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=True, parallel=False, timeout=2)

# Call query_control 6 times. We should expect 6 records(excluding NaN) for each controller.
# In single thread mod, each call of query_control will trigger a computations for each controller in the system.
t = time.time()
ctrl_stack.run_query_control_for(20)
for df in ctrl_stack.log_to_df().values():
    display(df)
print(time.time() - t)

,a,b,c,Logging
2021-03-08 22:22:51.943815936,NaN,NaN,NaN,Initialize
2021-03-08 22:22:56.079161088,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:22:58.367414016,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:23:00.636984064,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:23:02.916833024,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:23:05.189573120,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:23:07.468496896,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:23:09.748220928,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:23:12.044268032,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:23:14.320057088,10.0,4.0,40.0,testcontroller1 did a computation!


,a,b,c,Logging
2021-03-08 22:22:51.943815936,NaN,NaN,NaN,Initialize
2021-03-08 22:22:56.079161088,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:22:58.367414016,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:23:00.636984064,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:23:02.916833024,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:23:05.189573120,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:23:07.468496896,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:23:09.748220928,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:23:12.044268032,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:23:14.320057088,40.0,10.0,400.0,testcontroller2 did a computation!


,a,b,c,Logging
2021-03-08 22:22:51.943815936,NaN,NaN,NaN,Initialize
2021-03-08 22:22:56.079161088,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:22:58.367414016,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:23:00.636984064,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:23:02.916833024,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:23:05.189573120,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:23:07.468496896,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:23:09.748220928,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:23:12.044268032,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:23:14.320057088,400.0,40.0,16000.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-03-08 22:22:51.943815936,NaN,NaN,NaN,Initialize
2021-03-08 22:22:56.079161088,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:22:58.367414016,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:23:00.636984064,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:23:02.916833024,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:23:05.189573120,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:23:07.468496896,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:23:09.748220928,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:23:12.044268032,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:23:14.320057088,20.0,4.0,80.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-03-08 22:22:51.943815936,NaN,NaN,NaN,Initialize
2021-03-08 22:22:56.079161088,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:22:58.367414016,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:23:00.636984064,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:23:02.916833024,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:23:05.189573120,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:23:07.468496896,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:23:09.748220928,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:23:12.044268032,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:23:14.320057088,30.0,4.0,120.0,testcontroller1 did a computation!


22.678214073181152


## Multi Thread

In [7]:
# create a mapping of controllers and their sample times
controllers = {}
controllers['forecast1'] = {'fun':testcontroller1, 'sampletime':1}
controllers['mpc1'] = {'fun':testcontroller2, 'sampletime':'forecast1'}
controllers['control1'] = {'fun':testcontroller3, 'sampletime':'mpc1'}
controllers['forecast2'] = {'fun':testcontroller3, 'sampletime':1}
controllers['forecast3'] = {'fun':testcontroller1, 'sampletime':1}

# Create a mapping of inputs for each controller
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=True, parallel=True, timeout=2)


# Call query_control 6 times. We should expect there are 6 records(excluding NaN) for each task.
# In multi thread mod, each call of query_control will trigger a computation for one controller within each task. 
# We assign tasks based on input dependency. Since the inputs of mpc1 and control1 depend on the out put of
# forecast1, they are in the same task as forecast1.
t = time.time()
ctrl_stack.run_query_control_for(50)
for df in ctrl_stack.log_to_df().values():
    display(df)
print(time.time() - t)

,a,b,c,Logging
2021-03-08 22:22:51.943815936,NaN,NaN,NaN,Initialize
2021-03-08 22:23:20.854192128,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:23:21.894906880,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:23:22.898629120,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:23:23.940237056,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:23:24.978796032,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:23:26.011842048,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:23:27.061583872,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:23:28.090889984,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:23:29.132431872,10.0,4.0,40.0,testcontroller1 did a computation!


,a,b,c,Logging
2021-03-08 22:22:51.943815936,NaN,NaN,NaN,Initialize
2021-03-08 22:23:20.854192128,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:23:21.894906880,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:23:22.898629120,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:23:23.940237056,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:23:24.978796032,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:23:26.011842048,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:23:27.061583872,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:23:28.090889984,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:23:29.132431872,40.0,10.0,400.0,testcontroller2 did a computation!


,a,b,c,Logging
2021-03-08 22:22:51.943815936,NaN,NaN,NaN,Initialize
2021-03-08 22:23:20.854192128,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:23:21.894906880,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:23:23.940237056,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:23:24.978796032,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:23:26.011842048,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:23:27.061583872,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:23:28.090889984,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:23:29.132431872,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:23:30.165479936,400.0,40.0,16000.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-03-08 22:22:51.943815936,NaN,NaN,NaN,Initialize
2021-03-08 22:23:20.854192128,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:23:21.894906880,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:23:23.940237056,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:23:24.978796032,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:23:26.011842048,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:23:27.061583872,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:23:29.132431872,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:23:30.165479936,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:23:31.211191040,20.0,4.0,80.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-03-08 22:22:51.943815936,NaN,NaN,NaN,Initialize
2021-03-08 22:23:20.854192128,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:23:21.894906880,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:23:22.898629120,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:23:23.940237056,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:23:24.978796032,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:23:26.011842048,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:23:27.061583872,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:23:28.090889984,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:23:29.132431872,30.0,4.0,120.0,testcontroller1 did a computation!


52.48575210571289


In [5]:
#Adding timeoutcontroller
controllers['timeout'] = {'fun': timeoutcontroller, 'sampletime': 5}

# Create a mapping of inputs for each controller
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'
# Initialize the controller_stack using the mappings above
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=True, parallel=True, timeout=2)
print("done initializing")

# Call query_control 6 times. We should expect 6 records(excluding NaN) for each controller.
# In single thread mod, each call of query_control will trigger a computations for each controller in the system.
t = time.time()
ctrl_stack.run_query_control_for(20)
for df in ctrl_stack.log_to_df().values():
    display(df)
print(time.time() - t)

done initializing
Controller timeout timeout


/Users/deoxi/FMLC/FMLC/stackedclasses.py:287: Warning: Controller timeout timeout
  warnings.warn('Controller {} timeout'.format(name), Warning)


Controller timeout timeout
Controller timeout timeout
Controller timeout timeout


,a,b,c,Logging
2021-03-08 22:29:52.717533952,NaN,NaN,NaN,Initialize
2021-03-08 22:30:03.676968960,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:30:04.718792960,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:30:05.761614080,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:30:06.811237888,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:30:07.861688064,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:30:08.868992000,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:30:09.871037952,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:30:10.924419072,10.0,4.0,40.0,testcontroller1 did a computation!
2021-03-08 22:30:11.945048064,10.0,4.0,40.0,testcontroller1 did a computation!


,a,b,c,Logging
2021-03-08 22:29:52.717533952,NaN,NaN,NaN,Initialize
2021-03-08 22:30:03.676968960,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:30:04.718792960,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:30:05.761614080,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:30:06.811237888,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:30:07.861688064,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:30:08.868992000,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:30:09.871037952,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:30:10.924419072,40.0,10.0,400.0,testcontroller2 did a computation!
2021-03-08 22:30:11.945048064,40.0,10.0,400.0,testcontroller2 did a computation!


,a,b,c,Logging
2021-03-08 22:29:52.717533952,NaN,NaN,NaN,Initialize
2021-03-08 22:30:03.676968960,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:30:04.718792960,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:30:05.761614080,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:30:06.811237888,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:30:07.861688064,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:30:09.871037952,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:30:10.924419072,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:30:12.966381056,400.0,40.0,16000.0,testcontroller3 did a computation!
2021-03-08 22:30:14.006619904,400.0,40.0,16000.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-03-08 22:29:52.717533952,NaN,NaN,NaN,Initialize
2021-03-08 22:30:03.676968960,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:30:04.718792960,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:30:05.761614080,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:30:06.811237888,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:30:08.868992000,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:30:10.924419072,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:30:12.966381056,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:30:14.006619904,20.0,4.0,80.0,testcontroller3 did a computation!
2021-03-08 22:30:15.051962880,20.0,4.0,80.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-03-08 22:29:52.717533952,NaN,NaN,NaN,Initialize
2021-03-08 22:30:03.676968960,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:30:04.718792960,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:30:05.761614080,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:30:06.811237888,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:30:07.861688064,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:30:08.868992000,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:30:09.871037952,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:30:10.924419072,30.0,4.0,120.0,testcontroller1 did a computation!
2021-03-08 22:30:11.945048064,30.0,4.0,120.0,testcontroller1 did a computation!


,a,Logging
2021-03-08 22:29:52.717533952,NaN,Initialize
2021-03-08 22:30:03.676968960,seen due to late computation,Compute ok
2021-03-08 22:30:08.711700992,seen due to late computation,Compute ok
2021-03-08 22:30:13.747479040,seen due to late computation,Compute ok
2021-03-08 22:30:18.775074048,seen due to late computation,Compute ok


22.336485862731934
